In [2]:
using Pkg; Pkg.activate("C:\\Users\\KarlH\\Desktop\\ComputationalPhysicsMaster\\Code\\OrbitalNeuralMethods")
using Revise
using OrbitalNeuralMethods

  Activating project at `C:\Users\KarlH\Desktop\ComputationalPhysicsMaster\Code\OrbitalNeuralMethods`
┌ Info: Precompiling OrbitalNeuralMethods [6de8678a-dd1c-461e-a987-24da1d9562b4]
└ @ Base loading.jl:1423


In [3]:
using BenchmarkTools: @btime

## HO Basis and Spatial System

In [4]:
l = 10
ω = 0.25
basis = SpinBasis(HOBasis(l, ω))

n = 2
V = ShieldedCoulomb(0.25)
grid = [x for x in range(-10, stop = 10, length = 2001)]
@time system = System(n, basis, grid, V);

  2.554066 seconds (3.91 M allocations: 208.623 MiB, 13.68% gc time, 98.51% compilation time)


In [5]:
system.spfs

20-element Vector{Vector{Float64}}:
 [1.9793222660179253e-6, 2.0294036800343063e-6, 2.0807002510646667e-6, 2.133240097022943e-6, 2.187051951768289e-6, 2.2421651777487688e-6, 2.2986097788845167e-6, 2.3564164136946345e-6, 2.4156164086717675e-6, 2.476241771908695e-6  …  2.476241771908695e-6, 2.4156164086717675e-6, 2.3564164136946345e-6, 2.2986097788845167e-6, 2.2421651777487688e-6, 2.187051951768289e-6, 2.133240097022943e-6, 2.0807002510646667e-6, 2.0294036800343063e-6, 1.9793222660179253e-6]
 [1.9793222660179253e-6, 2.0294036800343063e-6, 2.0807002510646667e-6, 2.133240097022943e-6, 2.187051951768289e-6, 2.2421651777487688e-6, 2.2986097788845167e-6, 2.3564164136946345e-6, 2.4156164086717675e-6, 2.476241771908695e-6  …  2.476241771908695e-6, 2.4156164086717675e-6, 2.3564164136946345e-6, 2.2986097788845167e-6, 2.2421651777487688e-6, 2.187051951768289e-6, 2.133240097022943e-6, 2.0807002510646667e-6, 2.0294036800343063e-6, 1.9793222660179253e-6]
 [-1.3995921964547984e-5, -1.4335700988132753e

## CCD

In [304]:
ccd = setup_CCD(system, α = 0.1);

In [327]:
for i in 1:10
    CCD_Update!(ccd)
end

In [328]:
energy(ccd)

1.0188052915143921

## CCSD

In [336]:
ccsd = setup_CCSD(system, α = 0.5);

In [347]:
for i in 1:10
    CCSD_Update!(ccsd)
end

In [348]:
energy(ccsd)

0.9297453185653959

In [221]:
corr_energy(ccsd)

-0.24294852395444164

# HF

In [39]:
hf = setup_HF(system);

In [34]:
HF_update!(hf, iters = 10);

In [40]:
energy(hf)

1.3836526203846078

In [23]:
system = System(hf);

## RHF

In [194]:
rhf = setup_RHF(system);

In [195]:
RHF_update!(rhf, iters = 10);

In [196]:
energy(rhf)

1.1795819853618934

In [197]:
system = System(rhf);